In [1]:
!conda install ffmpeg -y

Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [2]:
!pip install fastapi uvicorn python-multipart moviepy openai-whisper

In [ ]:
# 한국어 최적화된 STT 서버 (Large 모델 + 추가 최적화)
from fastapi import FastAPI, File, UploadFile
from moviepy import VideoFileClip
import whisper
import torch
import tempfile
import nest_asyncio
import uvicorn
import traceback
import subprocess
import os
import gc
app = FastAPI()
# GPU 메모리 최적화
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    torch.cuda.empty_cache()
# Large 모델 로딩 (더 정확하지만 느림)
print("Whisper Large 모델 로딩 중... (시간이 걸릴 수 있습니다)")
model = whisper.load_model("large", device=device)
print(f"모델 로딩 완료! 디바이스: {device}")
@app.post("/process_video")
async def process_video(file: UploadFile = File(...)):
    print("process_video 시작")
    try:
        # 업로드 파일명/확장자 확인
        filename = (file.filename or "").lower()
        if not filename.endswith((".mp4", ".webm", ".avi", ".mov")):
            return {"status": "error", "message": "mp4, webm, avi, mov 파일만 지원합니다."}
        ext = filename.split('.')[-1]
        ext = f".{ext}"
        # 업로드 파일을 메모리에서 읽기
        video_bytes = await file.read()
        # 파일 크기 제한 (100MB)
        if len(video_bytes) > 100 * 1024 * 1024:
            return {"status": "error", "message": "파일 크기는 100MB 이하여야 합니다."}
        # 임시로 업로드 확장자에 맞춰 저장
        with tempfile.NamedTemporaryFile(suffix=ext, delete=False) as temp_video:
            temp_video.write(video_bytes)
            video_path = temp_video.name
        audio_path = None
        # 1) MoviePy로 오디오 추출 시도
        try:
            with VideoFileClip(video_path) as video_clip:
                audio_clip = video_clip.audio
                if audio_clip is None:
                    if os.path.exists(video_path):
                        os.unlink(video_path)
                    return {"status": "error", "message": "동영상에서 음성을 찾을 수 없습니다."}
                # Whisper 최적화 오디오 포맷
                with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
                    audio_path = temp_audio.name
                audio_clip.write_audiofile(
                    audio_path,
                    codec="pcm_s16le",  # PCM 16bit
                    fps=16000,          # 16kHz (Whisper 최적)
                    ffmpeg_params=["-ac", "1"],  # mono
                    verbose=False,
                    logger=None,
                )
                audio_clip.close()
        except Exception as moviepy_err:
            # 2) ffmpeg 백업 방식
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
                audio_path = temp_audio.name
            cmd = [
                "ffmpeg", "-y", "-i", video_path,
                "-vn",                    # 비디오 제거
                "-acodec", "pcm_s16le",  # PCM s16
                "-ar", "16000",          # 16kHz
                "-ac", "1",              # mono
                "-af", "volume=1.5",     # 볼륨 약간 증가
                audio_path,
            ]
            try:
                result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            except subprocess.CalledProcessError as ffmpeg_err:
                # 임시 파일 정리
                if os.path.exists(video_path):
                    os.unlink(video_path)
                if os.path.exists(audio_path):
                    os.unlink(audio_path)
                return {
                    "status": "error",
                    "message": f"오디오 추출 실패: {moviepy_err}",
                }
        # 3) Whisper로 고품질 한국어 변환
        print("Whisper 변환 시작...")
        # GPU 메모리 정리
        if device == "cuda":
            torch.cuda.empty_cache()
        result = model.transcribe(
            audio_path,
            language="ko",                    # 한국어 지정
            fp16=(device == "cuda"),         # GPU 사용시 FP16
            temperature=0.0,                 # 더 일관된 결과
            beam_size=5,                     # 빔 서치 크기 증가
            best_of=5,                       # 최고 결과 선택
            no_speech_threshold=0.6,         # 무음 구간 임계값
            logprob_threshold=-1.0,          # 로그 확률 임계값
            compression_ratio_threshold=2.4,  # 압축 비율 임계값
            condition_on_previous_text=True, # 이전 텍스트 조건부
            verbose=False,
        )
        transcribed_text = (result.get("text") or "").strip()
        # 한국어 후처리
        if transcribed_text:
            import re
            # 한글, 공백, 숫자, 기본 문장부호만 허용
            filtered_text = re.sub(r'[^\uAC00-\uD7A3\s0-9.,!?~\-()"]', '', transcribed_text)
            # 연속 공백 제거
            filtered_text = re.sub(r'\s+', ' ', filtered_text)
            transcribed_text = filtered_text.strip()
        # 세그먼트 정보도 함께 반환 (선택사항)
        segments = []
        if result.get("segments"):
            for segment in result["segments"][:5]:  # 처음 5개만
                segments.append({
                    "start": round(segment.get("start", 0), 2),
                    "end": round(segment.get("end", 0), 2),
                    "text": segment.get("text", "").strip()
                })
        # 임시 파일 정리
        if os.path.exists(video_path):
            os.unlink(video_path)
        if audio_path and os.path.exists(audio_path):
            os.unlink(audio_path)
        # 메모리 정리
        if device == "cuda":
            torch.cuda.empty_cache()
        gc.collect()
        # 결과 반환
        response = {
            "status": "success",
            "transcription": transcribed_text,
            "language": "korean",
            "model": "whisper-large"
        }
        # 세그먼트 정보 추가 (선택사항)
        if segments:
            response["segments"] = segments
        return response
    except Exception as e:
        error_traceback = traceback.format_exc()
        print(f"Error: {e}")
        print(f"Traceback: {error_traceback}")
        # 예외 발생 시 임시 파일 정리
        try:
            if 'video_path' in locals() and os.path.exists(video_path):
                os.unlink(video_path)
            if 'audio_path' in locals() and audio_path and os.path.exists(audio_path):
                os.unlink(audio_path)
        except:
            pass
        # GPU 메모리 정리
        if device == "cuda":
            torch.cuda.empty_cache()
        gc.collect()
        return {
            "status": "error",
            "message": str(e),
            "traceback": error_traceback
        }
@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "model": "whisper-large",
        "language": "korean",
        "device": device
    }
@app.get("/model-info")
async def model_info():
    return {
        "model_name": "whisper-large",
        "device": device,
        "language": "korean",
        "supported_formats": ["mp4", "webm", "avi", "mov"],
        "max_file_size": "100MB"
    }
# Jupyter notebook에서 서버 실행
# 서버 시작 후 다음 주소로 접속:
# - API 문서: http://localhost:8000/docs
# - 헬스체크: http://localhost:8000/health
# - 모델정보: http://localhost:8000/model-info
#
# 사용 방법:
# 1. http://localhost:8000/docs 접속
# 2. /process_video 섹션에서 "Try it out" 버튼 클릭
# 3. "Choose File"로 MP4/WebM 파일 선택
# 4. "Execute" 버튼 클릭
# 5. Response body에서 한국어 변환 결과 확인

try:
    nest_asyncio.apply()
    print("서버 시작 중...")
    uvicorn.run(app, host="0.0.0.0", port=8000, http='h11')
except RuntimeError:
    pass

Whisper Large 모델 로딩 중... (시간이 걸릴 수 있습니다)


INFO:     Started server process [12404]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


모델 로딩 완료! 디바이스: cuda
서버 시작 중...
INFO:     127.0.0.1:51099 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51099 - "GET /openapi.json HTTP/1.1" 200 OK
process_video 시작
Whisper 변환 시작...


100%|███████████████████████████████████████████████████████████████████████████| 592/592 [00:02<00:00, 293.44frames/s]

INFO:     127.0.0.1:51114 - "POST /process_video HTTP/1.1" 200 OK
